In [1]:
import glob
import pandas as pd
import os
import sys

from bs4 import BeautifulSoup
from collections import deque
from dateutil import parser
from IPython.core.debugger import set_trace
from tqdm import tqdm

sys.path.append('../..')

In [2]:
MESSAGES_DIR = 'source/messages'
AUTHOR = 'Pavel Korytov'

MONTHS = 'янв', 'фев', 'мар', 'апр', 'мая', 'июн', \
    'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'

In [3]:
def parse_date_russian(date):
    is_edited = False
    if date.endswith(' (ред.)'):
        is_edited = True
        date = date[:-7]
    day, month, year, _, time = date.split()
        
    month = MONTHS.index(month) + 1
    date = parser.parse(time).replace(day=int(day), month=month, year=int(year))
    return is_edited, date

In [4]:
def parse_date_english(date):
    is_edited = False
    if date.endswith(' (edited)'):
        is_edited = True
        date = date[:-9]
    date = date[2:]
    date = parser.parse(date)
    return is_edited, date

parse_date_english('at 5:52:25 pm on Dec 29, 2019 (edited)')

(True, datetime.datetime(2019, 12, 29, 17, 52, 25))

In [5]:
def parse_file(path):
    with open(path, 'r', encoding='windows-1251') as file:
        soup = BeautifulSoup(file)
        
        content = soup.html.body.div
        name = content.find(class_='page_content page_block').h2.div.find(class_='_header_inner') \
            .find('div', class_='ui_crumb').text
        items = content.find(class_='page_content page_block').find(class_='wrap_page_content')
        
        senders, recipients, dates, messages, edited = deque(), deque(), deque(), deque(), deque()
        
        for item in items.find_all(class_='item'):
            header = item.div.find(class_='message__header')
            author, date = header.text.split(', ', 1)
            if author == 'You':
                author, recipient = AUTHOR, name
            else:
                recipient = AUTHOR
            
            is_edited, date = parse_date_english(date)
            message_div = item.div.find('div', class_='')
            message = ''
            for content in message_div.contents:
                if content.name is None:
                    if message:
                        message += '\n' + content
                    else:
                        message += content
            
            if message:
                senders.append(author)
                recipients.append(recipient)
                dates.append(date)
                messages.append(message)
                edited.append(is_edited)
        
        return pd.DataFrame({
            "sender": senders,
            "recepient": recipients,
            "message": messages,
            "date": dates,
            "is_edited": edited
        }), name

df, name = parse_file('source/messages/2000000010/messages0.html')
display(df)
display(name)

,sender,recepient,message,date,is_edited
0,Pavel Korytov,Электрон-2044 Третий глаз,В браузере Vieb\nhttps://github.com/Jelmerro/V...,2020-06-05 18:55:39,False
1,Pavel Korytov,Электрон-2044 Третий глаз,https://drive.google.com/file/d/1EICYd3q3XG8XD...,2020-05-30 14:58:25,False
2,Pavel Korytov,Электрон-2044 Третий глаз,Ему\nИ заявление там ещё,2020-05-30 14:58:07,False
3,Yaroslav Piskunov,Pavel Korytov,"Ему короче, да?",2020-05-30 14:57:37,False
4,Pavel Korytov,Электрон-2044 Третий глаз,"В новом документе сказано, Родионову посылать",2020-05-30 14:57:23,False
5,Yaroslav Piskunov,Pavel Korytov,В ту форму или Генриховне на почту?,2020-05-30 14:57:03,False
6,Pavel Korytov,Электрон-2044 Третий глаз,Отправил на нормконтроль записку,2020-05-30 14:56:16,False
7,Pavel Korytov,Электрон-2044 Третий глаз,https://www.reddit.com/r/mapporn/comments/grsj...,2020-05-28 09:06:20,False
8,Pavel Korytov,Электрон-2044 Третий глаз,"У меня много пересечений будет, наверное",2020-05-27 10:58:35,False
9,Pavel Korytov,Электрон-2044 Третий глаз,"Мне удачно, что по неопубликованным статьям у ...",2020-05-27 10:57:56,False


'Электрон-2044 Третий глаз'

In [6]:
def fix_group(df, name):
    recepients = df.recepient.unique()
    df.sender = df.sender.apply(lambda sender: AUTHOR if sender == AUTHOR else name)
    return df

fix_group(df, name)

,sender,recepient,message,date,is_edited
0,Pavel Korytov,Электрон-2044 Третий глаз,В браузере Vieb\nhttps://github.com/Jelmerro/V...,2020-06-05 18:55:39,False
1,Pavel Korytov,Электрон-2044 Третий глаз,https://drive.google.com/file/d/1EICYd3q3XG8XD...,2020-05-30 14:58:25,False
2,Pavel Korytov,Электрон-2044 Третий глаз,Ему\nИ заявление там ещё,2020-05-30 14:58:07,False
3,Электрон-2044 Третий глаз,Pavel Korytov,"Ему короче, да?",2020-05-30 14:57:37,False
4,Pavel Korytov,Электрон-2044 Третий глаз,"В новом документе сказано, Родионову посылать",2020-05-30 14:57:23,False
5,Электрон-2044 Третий глаз,Pavel Korytov,В ту форму или Генриховне на почту?,2020-05-30 14:57:03,False
6,Pavel Korytov,Электрон-2044 Третий глаз,Отправил на нормконтроль записку,2020-05-30 14:56:16,False
7,Pavel Korytov,Электрон-2044 Третий глаз,https://www.reddit.com/r/mapporn/comments/grsj...,2020-05-28 09:06:20,False
8,Pavel Korytov,Электрон-2044 Третий глаз,"У меня много пересечений будет, наверное",2020-05-27 10:58:35,False
9,Pavel Korytov,Электрон-2044 Третий глаз,"Мне удачно, что по неопубликованным статьям у ...",2020-05-27 10:57:56,False


In [7]:
from api import DBConn
from models import VkMessage, VkUser, Base

DBConn()

DBConn.engine.execute('DROP SCHEMA IF EXISTS vk CASCADE')
DBConn.engine.execute('CREATE SCHEMA vk')
Base.metadata.create_all(DBConn.engine)

In [8]:
def parse_directory(path):
    files = sorted([file for file in os.listdir(path) if file.endswith('html')])
    df = pd.DataFrame(columns=['sender', 'recipient', 'message', 'date', 'is_edited'])
    for file in tqdm(files, desc=path):
        df_, name = parse_file(os.path.join(path, file))
        df = pd.concat([df, df_])
    df = df.sort_values(by='date').reset_index(drop=True)
    return df, name



for file in os.listdir(MESSAGES_DIR):
    path = os.path.join(MESSAGES_DIR, file)
    if os.path.isdir(path):
        id_ = file
        if path.endswith('.ipynb_checkpoints'):
            continue
        df, name = parse_directory(path)
        
        df['target_id'] = id_
        is_group = df.sender.nunique() > 2
        if is_group:
            df = fix_group(df, name)
        with DBConn.get_session() as db:
            user = VkUser(name=name, id=id_, is_group=is_group)
            data = df.to_dict(orient='records')
            db.add(user)
            db.commit()
            
            db.bulk_insert_mappings(VkMessage, data)
            db.commit()

source/messages/16096886: 100%|██████████| 1/1 [00:00<00:00, 55.21it/s]
